# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../WeatherPY/output_data/weather_city_data.csv"
weather_result = pd.read_csv(cities_csv)
weather_result

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,grindavik,63.8424,-22.4338,46.40,71,90,10.36,IS,1619535558
1,port alfred,-33.5906,26.8910,69.64,66,0,18.70,ZA,1619535559
2,aras,42.5617,-2.3560,57.99,82,75,5.75,ES,1619535559
3,georgetown,5.4112,100.3354,82.40,83,40,2.30,MY,1619535559
4,rikitea,-23.1203,-134.9692,76.17,75,42,8.34,PF,1619535289
...,...,...,...,...,...,...,...,...,...
565,praia,14.9215,-23.5087,78.80,57,75,19.57,CV,1619535825
566,dillon,34.4166,-79.3712,73.99,46,1,5.75,US,1619535826
567,xiaoweizhai,26.1903,107.5125,58.75,94,38,1.90,CN,1619535592
568,midland,31.9974,-102.0779,78.01,50,1,11.50,US,1619535615


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_result[["Lat","Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_result["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=np.max(humidity), 
                                 point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Option 1

weather_criteria = weather_result.loc[(weather_result["Wind Speed"] <= 10) &
                                         (weather_result["Cloudiness"] == 0) &
                                         (weather_result["Max Temp"] >= 70) & 
                                         (weather_result["Max Temp"] <= 80)]

# Remove the NaN Values

weather_criteria = weather_criteria.dropna()

# Reset Index

weather_criteria = weather_criteria.reset_index()
weather_criteria


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,10,mount isa,-20.7333,139.5000,75.20,25,0,5.75,AU,1619535561
1,86,suluktu,39.9337,69.5700,77.00,27,0,3.98,KG,1619535592
2,104,imbituba,-28.2400,-48.6703,71.60,56,0,4.61,BR,1619535612
3,179,gat,31.6100,34.7642,70.00,53,0,1.99,IL,1619535480
4,230,san ignacio,17.1561,-89.0714,78.80,73,0,6.91,BZ,1619535673
5,270,karratha,-20.7377,116.8463,78.94,34,0,9.13,AU,1619535691
6,315,noumea,-22.2763,166.4572,75.00,77,0,2.30,NC,1619535605
7,328,san juan,-31.5375,-68.5364,71.33,41,0,3.62,AR,1619535719
8,377,roebourne,-20.7833,117.1333,78.84,33,0,8.32,AU,1619535741
9,520,port hedland,-20.3167,118.5667,73.40,49,0,8.05,AU,1619535806


In [6]:
# Option 2 - different way to solve it.

weather_criteria2 = weather_result.loc[(weather_result["Wind Speed"] <= 10) & (weather_result["Cloudiness"]== 0) & \
                  (weather_result["Max Temp"] >= 70) & (weather_result["Max Temp"] <= 80)].dropna()
weather_criteria2

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,mount isa,-20.7333,139.5000,75.20,25,0,5.75,AU,1619535561
86,suluktu,39.9337,69.5700,77.00,27,0,3.98,KG,1619535592
104,imbituba,-28.2400,-48.6703,71.60,56,0,4.61,BR,1619535612
179,gat,31.6100,34.7642,70.00,53,0,1.99,IL,1619535480
230,san ignacio,17.1561,-89.0714,78.80,73,0,6.91,BZ,1619535673
270,karratha,-20.7377,116.8463,78.94,34,0,9.13,AU,1619535691
315,noumea,-22.2763,166.4572,75.00,77,0,2.30,NC,1619535605
328,san juan,-31.5375,-68.5364,71.33,41,0,3.62,AR,1619535719
377,roebourne,-20.7833,117.1333,78.84,33,0,8.32,AU,1619535741
520,port hedland,-20.3167,118.5667,73.40,49,0,8.05,AU,1619535806


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create hotel_df with hotel name column

hotel_df = weather_criteria

# Add a "Hotel Name" column to the DF

hotel_df["Hotel Name"] = ""
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,10,mount isa,-20.7333,139.5000,75.20,25,0,5.75,AU,1619535561,
1,86,suluktu,39.9337,69.5700,77.00,27,0,3.98,KG,1619535592,
2,104,imbituba,-28.2400,-48.6703,71.60,56,0,4.61,BR,1619535612,
3,179,gat,31.6100,34.7642,70.00,53,0,1.99,IL,1619535480,
4,230,san ignacio,17.1561,-89.0714,78.80,73,0,6.91,BZ,1619535673,
5,270,karratha,-20.7377,116.8463,78.94,34,0,9.13,AU,1619535691,
6,315,noumea,-22.2763,166.4572,75.00,77,0,2.30,NC,1619535605,
7,328,san juan,-31.5375,-68.5364,71.33,41,0,3.62,AR,1619535719,
8,377,roebourne,-20.7833,117.1333,78.84,33,0,8.32,AU,1619535741,
9,520,port hedland,-20.3167,118.5667,73.40,49,0,8.05,AU,1619535806,


In [8]:
 #Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
# Using the example from class - Nearest Restaurant.
# Find the closest hotel of each type to coordinates.

# Get city coordinates from DF.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
      
    # Add keyword to params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request.
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract Results
    results = response['results']
    
    # Save the hotel name into the Dataframe.
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Could not find result")
        
        print("-------------------------")

Retrieving Results for Index 0: mount isa.
Closest hotel in mount isa is Redearth Boutique Hotel.
Retrieving Results for Index 1: suluktu.
Could not find result
-------------------------
Retrieving Results for Index 2: imbituba.
Closest hotel in imbituba is Praia Hotel Imbituba.
Retrieving Results for Index 3: gat.
Closest hotel in gat is Desert Gat.
Retrieving Results for Index 4: san ignacio.
Closest hotel in san ignacio is San Ignacio Resort Hotel.
Retrieving Results for Index 5: karratha.
Closest hotel in karratha is Karratha International Hotel.
Retrieving Results for Index 6: noumea.
Closest hotel in noumea is Hilton Noumea La Promenade Residences.
Retrieving Results for Index 7: san juan.
Closest hotel in san juan is Hotel Viñas del Sol.
Retrieving Results for Index 8: roebourne.
Closest hotel in roebourne is Latitude20 Roebourne Village.
Retrieving Results for Index 9: port hedland.
Closest hotel in port hedland is Hedland Hotel.
Retrieving Results for Index 10: moron.
Closest 

In [10]:
# # Fill the DF with the response get from the JSON with the hotels' name.
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,10,mount isa,-20.7333,139.5000,75.20,25,0,5.75,AU,1619535561,Redearth Boutique Hotel
1,86,suluktu,39.9337,69.5700,77.00,27,0,3.98,KG,1619535592,
2,104,imbituba,-28.2400,-48.6703,71.60,56,0,4.61,BR,1619535612,Praia Hotel Imbituba
3,179,gat,31.6100,34.7642,70.00,53,0,1.99,IL,1619535480,Desert Gat
4,230,san ignacio,17.1561,-89.0714,78.80,73,0,6.91,BZ,1619535673,San Ignacio Resort Hotel
5,270,karratha,-20.7377,116.8463,78.94,34,0,9.13,AU,1619535691,Karratha International Hotel
6,315,noumea,-22.2763,166.4572,75.00,77,0,2.30,NC,1619535605,Hilton Noumea La Promenade Residences
7,328,san juan,-31.5375,-68.5364,71.33,41,0,3.62,AR,1619535719,Hotel Viñas del Sol
8,377,roebourne,-20.7833,117.1333,78.84,33,0,8.32,AU,1619535741,Latitude20 Roebourne Village
9,520,port hedland,-20.3167,118.5667,73.40,49,0,8.05,AU,1619535806,Hedland Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))